In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df_for_ship_class = pd.read_csv("radar_dataset/df_for_ship_class.csv")

In [4]:
index_drop = []
for index, row in df_for_ship_class.iterrows():
    if (df_for_ship_class["marine_traffic_class"][index] == 'fishery_research vessel'):
        index_drop.append(index)

In [5]:
df_for_ship_class = df_for_ship_class.drop(index_drop)

In [29]:
df_for_ship_class

,np1,np2,np3,np4,np5,np6,np7,np8,np9,np10,...,np194,np195,np196,np197,np198,np199,np200,view_angle,distance_to_radar,marine_traffic_class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-165.621794,26609.081047,oil_products tanker
1,11,15,17,9,14,27,27,18,3,9,...,40,31,24,8,17,20,11,-43.317525,5163.925502,tug
2,5,5,4,4,5,6,6,7,7,7,...,0,0,0,0,0,0,0,131.641740,2655.661736,tug
3,9,13,4,3,5,8,13,18,16,8,...,0,0,0,0,0,0,0,-87.530959,15204.475378,yacht/pleasure_craft
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-165.604741,26581.600069,oil_products tanker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25.461893,32172.729858,oil/chemical_tanker
308892,0,0,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,0,23.209848,42977.750563,container_ship
308893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,43.401273,31280.847224,ro-ro_cargo
308894,2,3,4,9,8,4,8,7,10,8,...,17,13,21,23,18,18,16,46.741852,18777.002455,general_cargo


In [6]:
data_labels_for_training = []
for i in range(1, 201):
        data_labels_for_training.append("np" + str(i))
data_labels_for_training.append("view_angle")
data_labels_for_training.append("distance_to_radar")

target_label = "marine_traffic_class"

data = df_for_ship_class[data_labels_for_training].to_numpy()
labels = df_for_ship_class[target_label].to_numpy()

In [20]:
def get_classes_size(labels):
    classes = {}
    for i in labels.transpose():
        if i in classes.keys():
            classes[i]+=1
        else:
            classes[i]=1
    return classes

classes = get_classes_size(labels)
print(classes)

{'oil_products tanker': 13357, 'tug': 23164, 'yacht/pleasure_craft': 2262, 'container_ship': 41605, 'oil/chemical_tanker': 55696, 'ro-ro/passenger_ship': 21078, 'general_cargo': 18390, 'crude_oil tanker': 62286, 'hopper_dredger': 14098, 'pilot_ship': 4699, 'lpg_tanker': 5794, 'sailing_vessel': 2084, 'bulk_carrier': 2390, 'training_ship': 7986, 'other': 173, 'trawler': 3339, 'tanker': 1026, 'ro-ro_cargo': 1896, 'vehicles_carrier': 9680, 'chemical_tanker': 2138, 'passenger_ship': 6034, 'ro-ro/container_carrier': 786, 'high_speed craft': 283, 'fishing': 3328, 'passenger': 127, 'rescue/salvage_ship': 196, 'aggregates_carrier': 119, 'container': 303, 'asphalt/bitumen_tanker': 1029, 'fishing_vessel': 118, 'dry_cargo': 892, 'heavy_load carrier': 168, 'crewboat': 1616, 'lh896354': 755}


In [8]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, train_size=0.8)

pool = Pool(data, labels)
pool_train = Pool(train_data, train_labels)
pool_test = Pool(test_data, test_labels)

model = CatBoostClassifier(iterations=1500,
                           depth=3,
                           learning_rate=0.2,
                           loss_function='MultiClass',
                           task_type="GPU",
                           early_stopping_rounds = 20,
                           verbose=False)

In [9]:
model.fit(pool_train, eval_set=pool_test, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [10]:
preds_class_test = model.predict(pool_test)
preds_class_test = preds_class_test.transpose()
print(model.score(test_data, test_labels))

0.8056459314653847


In [11]:
from catboost import cv

In [12]:
params = {}
params['loss_function'] = 'MultiClass'
params['iterations'] = 1500
params['custom_loss'] = 'AUC'
params['learning_rate'] = 0.2
params['task_type'] = 'GPU'
params['depth'] = 3
params['early_stopping_rounds'] = 20

In [13]:
cv_data1 = cv(
    params = params,
    pool = pool,
    fold_count = 5,
    shuffle = True,
    partition_random_seed = 0,
    plot=True,
    stratified = True,
    verbose = False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.6472943183
bestIteration = 1499
Training on fold [1/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

bestTest = 0.6430696862
bestIteration = 1499
Training on fold [3/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.64565762
bestIteration = 1499
Training on fold [4/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.6446171364
bestIteration = 1499


In [16]:
def true_positive(dict_class, predicted_labels, true_labels):
    tp_dict = dict.fromkeys(dict_class.keys(), 0)
    for i in range(0, len(predicted_labels)):
        if predicted_labels[i] == true_labels[i]:
            tp_dict[true_labels[i]]+=1
    
    for k in tp_dict.keys():
        tp_dict[k] = tp_dict[k]/dict_class[k]
        
    return tp_dict

In [26]:
tp = true_positive(get_classes_size(test_labels), preds_class_test[0], test_labels)

for k in tp.keys():
    print(k + ": " + str(tp[k]))

oil_products tanker: 0.8344519015659956
yacht/pleasure_craft: 0.8004587155963303
training_ship: 0.7975460122699386
hopper_dredger: 0.6357711803041275
passenger_ship: 0.6972704714640199
tug: 0.855526544821584
crude_oil tanker: 0.9466933867735471
general_cargo: 0.6752659574468085
pilot_ship: 0.6788399570354458
oil/chemical_tanker: 0.8367291741795889
container_ship: 0.8613103196791054
bulk_carrier: 0.571992110453649
sailing_vessel: 0.7026378896882494
lpg_tanker: 0.24520069808027922
vehicles_carrier: 0.5903490759753593
ro-ro/passenger_ship: 0.8739751698289998
fishing: 0.760670731707317
lh896354: 0.3051948051948052
container: 0.16666666666666666
ro-ro_cargo: 0.3815789473684211
trawler: 0.9279538904899135
chemical_tanker: 0.33640552995391704
crewboat: 0.7049689440993789
tanker: 0.41784037558685444
passenger: 0.08
dry_cargo: 0.6309523809523809
high_speed craft: 0.25757575757575757
ro-ro/container_carrier: 0.5578231292517006
asphalt/bitumen_tanker: 0.8640776699029126
heavy_load carrier: 0.6052

In [35]:
print("Ship percentage")
temp = 0
for k in classes.keys():
    temp += classes[k]*100/df_for_ship_class.shape[0]
    print(k + ": " +str(classes[k]*100/df_for_ship_class.shape[0]))

Ship percentage
oil_products tanker: 4.324123083895822
tug: 7.4989883293675845
yacht/pleasure_craft: 0.7322876705676686
container_ship: 13.46897813172761
oil/chemical_tanker: 18.030722413765194
ro-ro/passenger_ship: 6.82367794881756
general_cargo: 5.953479337639004
crude_oil tanker: 20.1641334434031
hopper_dredger: 4.564010424254197
pilot_ship: 1.5212288965506078
lpg_tanker: 1.8757182861490151
sailing_vessel: 0.6746629113452791
bulk_carrier: 0.7737256996714094
training_ship: 2.5853445345505754
other: 0.056006086210524615
trawler: 1.0809498373233624
tanker: 0.332151702034672
ro-ro_cargo: 0.6138008060991599
vehicles_carrier: 3.1337509509703945
chemical_tanker: 0.6921445798734198
passenger_ship: 1.953414590718529
ro-ro/container_carrier: 0.25445539746515805
high_speed craft: 0.09161689247155182
fishing: 1.0773887566972595
passenger: 0.041114294501367776
rescue/salvage_ship: 0.06345198206510302
aggregates_carrier: 0.03852441768238398
container: 0.09809158451901132
asphalt/bitumen_tanker: 0

In [36]:
def false_positive_count(dict_class, predicted_labels, true_labels):
    fp_dict = dict.fromkeys(dict_class.keys(), 0)
    for i in range(0, len(predicted_labels)):
        if predicted_labels[i] != true_labels[i]:
            fp_dict[predicted_labels[i]]+=1
    
    #for k in tp_dict.keys():
    #    fp_dict[k] = fp_dict[k]/dict_class[k]
        
    return fp_dict

In [38]:
fp = false_positive_count(get_classes_size(test_labels), preds_class_test[0], test_labels)

for k in fp.keys():
    print(k + ": " + str(fp[k]))

oil_products tanker: 278
yacht/pleasure_craft: 135
training_ship: 471
hopper_dredger: 564
passenger_ship: 94
tug: 2044
crude_oil tanker: 712
general_cargo: 1095
pilot_ship: 296
oil/chemical_tanker: 2801
container_ship: 1540
bulk_carrier: 82
sailing_vessel: 113
lpg_tanker: 136
vehicles_carrier: 267
ro-ro/passenger_ship: 735
fishing: 205
lh896354: 29
container: 7
ro-ro_cargo: 26
trawler: 60
chemical_tanker: 92
crewboat: 99
tanker: 49
passenger: 2
dry_cargo: 11
high_speed craft: 6
ro-ro/container_carrier: 18
asphalt/bitumen_tanker: 20
heavy_load carrier: 3
rescue/salvage_ship: 5
other: 5
fishing_vessel: 4
aggregates_carrier: 3


Большое количество неправильных срабатываний на грузовых кораблях разных типов говорит о том, что либо надо собрать больше данных по остальным типам кораблей (так как грузовые разных видов преобладают в датасете), либо отдельно тренировать модель на то, чтобы она умела различать типы конкретно грузовых.

Можно было бы собрать типы кораблей по "надтипам" и сначала научиться различать их, а затем тренировать модели уже для каждого "надтипа" (например танкеры можно выделить в отдельную группу, однако я не сильно разбираюсь в кораблях, чтобы проводить такие объединения).

Плюс, тип научного судна и один из двух типов что был зашифрован цифровым кодом отсеялись, первый по причине того что одного корабля точно недостаточно для классификации, а у второго отсутствовал радарный профиль.

Еще можно вывести полную матрицу ложных срабатываний и посмотреть, какие корабли с какими путают чаще всего и смотреть в чём причина, но частично это можно оправдать несбалансированностью датасета.